In [3]:
import numpy as np
import pandas as pd
import customFunctions as fct
import foxes
import foxes.variables as FV
import foxes.input.farm_layout as layout
from foxes import algorithms
from foxes.output import FarmResultsEval
import os
os.environ["FOXES_ENGINE"]    = "threads"
os.environ["FOXES_N_PROCS"]   = "1"

# 1) Load & prepare ERA5 from reanalysis.csv
era5 = pd.read_csv("reanalysis.csv", index_col=0, parse_dates=True)
era5.rename(columns={"WS100":"WS","WD100":"WD"}, inplace=True)
era5.replace(-999, np.nan, inplace=True)
era5 = era5.resample("h").mean()
era5.index = pd.to_datetime(era5.index)
era5 = era5.loc["2006-01-01":"2006-12-31"]

# 2) Ensure turbulence intensity column exists
if "TI" not in era5.columns:
    era5["TI"] = 0.10  

# 3) Build FOXES TimeSeries states using the cleaned WS/WD
states = foxes.input.states.Timeseries(
    data_source=era5,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "WS", FV.WD: "WD", FV.TI: "TI"},
    fixed_vars={FV.RHO: 1.225},
)

# 4) Create the wind farm layout for N-10.1
farm = foxes.WindFarm(name="N-10.1")

layout_data = pd.read_csv(
    "turbine-info/coordinates/area_of_interest/layout-N-10.1.geom.csv",
    index_col=None
)
# add turbines using NREL5MW model
layout.add_from_csv(
    farm,
    layout_data,
    turbine_models=["NREL5MW"],
    verbosity=0
)

# 5) Set up & run FOXES (internal wake only)
algo = algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_quadratic_ka02"],
    verbosity=0
)

results = algo.calc_farm(
    calc_parameters={"chunk_size_states": 1000}
)

# 6) Aggregate results & compute yield/efficiency
from customFunctions import compute_yield

turbine_stats, summary = compute_yield(results, algo)
print(turbine_stats)
print(summary)

# %% 7) TASK 5 via setup_windfarm

from customFunctions import setup_windfarm, compute_yield

# Define the two sets of wake models to compare
wake_sets = [
    ["Bastankhah2014_quadratic_ka02", "CrespoHernandez_max_ka04"],
    ["Jensen1968",              "Frandsen1995"]
]

for wakes in wake_sets:
    print(f"\n=== Running wake models: {wakes} ===")
    
    # Re-use the same cleaned ERA5 DataFrame `era5` from above
    # setup_windfarm now returns (farm, algo)
    farm7, algo7 = setup_windfarm(
        wind_data    = era5,
        windfarm_name= "N-10.1",        # or any name you like
        TI           = 0.10,            # turbulence intensity
        RHO          = 1.225,           # air density
        wake_models  = wakes,           # pass your wake list
        model_book   = None             # use default ModelBook
    )
    
    # Run the calculation in chunks
    results7 = algo7.calc_farm(
        calc_parameters={"chunk_size_states": 1000}
    )
    
    # Compute per‐turbine stats & farm summary
    stats7, summary7 = compute_yield(results7, algo7)
    
    # Print farm‐level metrics
    print(f"Farm ambient power [MW]: {summary7['farm_ambient_power_MW']:.1f}")
    print(f"Farm net power     [MW]: {summary7['farm_net_power_MW']:.1f}")
    print(f"Farm efficiency        : {summary7['farm_efficiency']:.3f}")
    print(f"Annual yield      [GWh]: {summary7['annual_yield_GWh']:.2f}\n")
    
    # Show the first 5 turbine efficiencies
    print("Sample turbine efficiencies:")
    print(stats7["Efficiency         "].head(5))


DefaultEngine: Selecting engine 'process'
ProcessEngine: Calculating 8760 states for 133 turbines
ProcessEngine: Computing 12 chunks using 12 processes


100%|██████████| 12/12 [01:00<00:00,  5.01s/it]


Capacity added to farm results
Ambient capacity added to farm results
Efficiency added to farm results
      Ambient Yield [GWh]  Net Yield     [GWh]  Efficiency         
T0              24.755557            24.101155             0.973565
T1              24.755557            24.028297             0.970622
T2              24.755557            24.024398             0.970465
T3              24.755557            24.261753             0.980053
T4              24.755557            23.714143             0.957932
...                   ...                  ...                  ...
T128            24.755557            23.274699             0.940181
T129            24.755557            23.074191             0.932081
T130            24.755557            23.190648             0.936786
T131            24.755557            23.117964             0.933849
T132            24.755557            23.040115             0.930705

[133 rows x 3 columns]
{'farm_ambient_power_MW': np.float64(375.8549135178814), 

ImportError: cannot import name 'setup_windfarm' from 'customFunctions' (c:\Users\nuwaw\Documents\GitHub\Project_Python_Programming_in_Energy_Science\customFunctions.py)